In [1]:
import ipywidgets as widgets

# Create a widget
text_widget = widgets.Text(description='Enter your name:')
button_widget = widgets.Button(description='Greet')

# Define a callback function for the button click event
def greet_button_clicked(b):
    name = text_widget.value
    print(f"Hello, {name}!")

# Register the callback function to the button click event
button_widget.on_click(greet_button_clicked)

# Display the widgets
display(text_widget, button_widget)

Text(value='', description='Enter your name:')

Button(description='Greet', style=ButtonStyle())

In [2]:
import os
from voila import *
from flask import Flask, request, render_template
from werkzeug.utils import secure_filename

app = Flask(__name__)

# Configure the upload folder and allowed file extensions
UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = {'ipynb'}

# Set the upload folder for Flask app
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER


# Function to check if the file has an allowed extension
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS


# Route for the main page
@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        # Check if a file was uploaded
        if 'file' not in request.files:
            return render_template('home.html', error='No file uploaded')

        file = request.files['file']

        # Check if a file was selected
        if file.filename == '':
            return render_template('home.html', error='No file selected')

        # Check if the file has an allowed extension
        if not allowed_file(file.filename):
            return render_template('home.html', error='Invalid file format. Only .ipynb files are allowed.')

        # Save the file to the upload folder
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))

        # Redirect to the Voilà app for the uploaded notebook
        return Voila.show_notebook(f'uploads/{filename}')



In [3]:
import os
from flask import Flask, render_template, request, redirect, url_for
from werkzeug.utils import secure_filename
from subprocess import Popen, PIPE

app = Flask(__name__)
UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = {'ipynb'}

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        # Check if the post request has the file part
        if 'file' not in request.files:
            return redirect(request.url)
        file = request.files['file']
        # If the user does not select a file, the browser submits an empty file
        if file.filename == '':
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            p = Popen(['voila', f"uploads/{filename}"], stdout=PIPE, stderr=PIPE)
            p.wait()
            return p.stdout.read().decode('utf-8')
    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
 * Restarting with stat
Traceback (most recent call last):
  File "/home/hd/.local/lib/python3.9/site-packages/ipykernel_launcher.py", line 15, in <module>
    from ipykernel import kernelapp as app
  File "/home/hd/.local/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 18, in <module>
    from IPython.core.application import (
  File "/home/hd/.local/lib/python3.9/site-packages/IPython/__init__.py", line 52, in <module>
    from .terminal.embed import embed
  File "/home/hd/.local/lib/python3.9/site-packages/IPython/terminal/embed.py", line 17, in <module>
    from IPython.terminal.ipapp import load_default_config
  File "/home/hd/.local/lib/python3.9/site-packages/IPython/terminal/ipapp.py", line 29, in <module>
    from IPython.core.magics import (
  File "/home/hd/.local/lib/python3.9/site-packages/IPython/core/magics/__init__.py", line 21, in <module>
    from .execution import ExecutionMagics
  File "/home/hd/.l

SystemExit: 1

/home/hd/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
